# Instalação de pacotes

In [12]:
!powershell pip install \
            face_recognition \
            git+https://github.com/ageitgey/face_recognition_models \
            opencv-python \
            pillow \
            numpy \
            tqdm \
            qdrant-client \
            setuptools \
            ipywidgets \
            matplotlib \
            scikit-learn \
            scikit-image

  Cloning https://github.com/ageitgey/face_recognition_models to c:\users\despinxz\appdata\local\temp\pip-req-build-ht9vz2if
  Resolved https://github.com/ageitgey/face_recognition_models to commit e67de717267507d1e9246de95692eb8be736ab61
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --------------------- ------------------ 7.1/12.9 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------- 12

  Running command git clone --filter=blob:none --quiet https://github.com/ageitgey/face_recognition_models 'C:\Users\despinxz\AppData\Local\Temp\pip-req-build-ht9vz2if'


# Rodar QDrant

In [1]:
!powershell docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

^C


# Imports

In [2]:
import os
import uuid
import numpy as np
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog
import face_recognition
import face_recognition_models
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import Filter, SearchRequest
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
import skimage

# Configurações

In [3]:
def cria_cliente():
    # Conectando ao Qdrant local
    client = QdrantClient("http://localhost:6333")
    return client

def cria_colecao(client, nome_colecao, tamanho_vetor):
    # Cria a coleção se não existir
    if nome_colecao not in [c.name for c in client.get_collections().collections]:
        client.recreate_collection(
            collection_name=nome_colecao,
            vectors_config=VectorParams(size=tamanho_vetor, distance=Distance.COSINE),
        )

# Códigos para processar imagens

### Gerar vetor de características por Local Binary Patterns

In [4]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def extrai_caracteristicas_lbp(caminho_imagem):
    imagem = cv2.imread(caminho_imagem)
    # Converte imagem para tons de cinza
    imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    metodo = 'uniform'
    vetor_final = []

    # Para radius 1, 2 e 3
    for raio in [1, 2, 3]:
        n_pontos = 8 * raio

        # Aplica o LBP
        lbp = local_binary_pattern(imagem_cinza, n_pontos, raio, metodo)

        # Número de bins: n_pontos + 2 para o método 'uniform'
        n_bins = n_pontos + 2
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)

        # Adiciona o histograma ao vetor final
        vetor_final.extend(hist)

    id_ponto = str(uuid.uuid4())    # ID único 

    nome_pessoa = caminho_imagem.split("\\")[-1].split(".")[0]

    # Cria estrutura de ponto para inserir na coleção
    ponto = PointStruct(
            id=id_ponto,
            vector=vetor_final,
            payload={"nome": nome_pessoa, "arquivo": caminho_imagem})

    return ponto


### Gerar vetor de características por descritores faciais

In [46]:
import cv2
import numpy as np
import face_recognition

def alinhar_rosto_por_landmarks(imagem_bgr, tamanho_saida=(160, 160)):
    imagem_rgb = cv2.cvtColor(imagem_bgr, cv2.COLOR_BGR2RGB)
    landmarks_list = face_recognition.face_landmarks(imagem_rgb)

    if not landmarks_list:
        return None  # Nenhum rosto detectado

    landmarks = landmarks_list[0]

    # Pega posição dos olhos (centro de cada olho)
    olho_esquerdo = np.mean(landmarks['left_eye'], axis=0)
    olho_direito = np.mean(landmarks['right_eye'], axis=0)

    # Calcula ângulo de rotação
    dx = olho_direito[0] - olho_esquerdo[0]
    dy = olho_direito[1] - olho_esquerdo[1]
    angulo = np.degrees(np.arctan2(dy, dx))

    # Ponto central entre os olhos
    centro_olhos_np = (olho_esquerdo + olho_direito) / 2
    centro_olhos = (float(centro_olhos_np[0]), float(centro_olhos_np[1]))


    # Matriz de rotação
    M = cv2.getRotationMatrix2D(centro_olhos, angulo, scale=1.0)

    # Aplica rotação
    imagem_rotacionada = cv2.warpAffine(imagem_bgr, M, (imagem_bgr.shape[1], imagem_bgr.shape[0]))

    # Opcional: detectar novamente o rosto e centralizar / crop após rotação
    imagem_alinhada = cv2.resize(imagem_rotacionada, tamanho_saida)

    return imagem_alinhada


In [ ]:
import face_recognition
import numpy as np
import cv2

def extrai_caracteristicas_landmarks(caminho_imagem):
    imagem = cv2.imread(caminho_imagem)
    imagem_alinhada = alinhar_rosto_por_landmarks(imagem)

    if imagem_alinhada is None:
        vetor = np.zeros(136)    

    else: 
        imagem_rgb = cv2.cvtColor(imagem_alinhada, cv2.COLOR_BGR2RGB)
        # Detecta landmarks faciais
        face_landmarks_list = face_recognition.face_landmarks(imagem_rgb)
        if not face_landmarks_list:
            vetor = np.zeros(136)
        else:
            # Pegamos os landmarks da primeira face detectada
            landmarks = face_landmarks_list[0]

            # Reorganiza os pontos na ordem correta (68 pontos padrão)
            pontos = []
            for key in ['chin', 'left_eyebrow', 'right_eyebrow', 'nose_bridge', 'nose_tip',
                        'left_eye', 'right_eye', 'top_lip', 'bottom_lip']:
                for ponto in landmarks[key]:
                    pontos.append(ponto)

            # Garante que temos sempre 68 pontos
            if len(pontos) < 68:
                pontos += [(0, 0)] * (68 - len(pontos))

            # Normaliza por largura e altura da imagem
            h, w = imagem.shape[:2]
            vetor = []
            for (x, y) in pontos[:68]:
                vetor.append(x / w)
                vetor.append(y / h)

    id_ponto = str(uuid.uuid4())    # ID único 

    nome_pessoa = caminho_imagem.split("\\")[-1].split(".")[0]

    # Cria estrutura de ponto para inserir na coleção
    ponto = PointStruct(
            id=id_ponto,
            vector=vetor,
            payload={"nome": nome_pessoa, "arquivo": caminho_imagem})

    return ponto


### Gerar vetor de características por CNN

In [36]:
def extrai_caracteristicas_cnn(caminho_imagem):
    imagem = face_recognition.load_image_file(caminho_imagem)
    embeddings = face_recognition.face_encodings(imagem)
    if not embeddings:
        vetor = np.zeros(128)  # Nenhum rosto detectado
    else:
        vetor = embeddings[0]

    id_ponto = str(uuid.uuid4())    # ID único 

    nome_pessoa = caminho_imagem.split("\\")[-1].split(".")[0]

    # Cria estrutura de ponto para inserir na coleção
    ponto = PointStruct(
            id=id_ponto,
            vector=vetor,
            payload={"nome": nome_pessoa, "arquivo": caminho_imagem})

    return ponto

# Código para selecionar diretórios e separar conjuntos de treino e validação

In [7]:
import os
import random
from sklearn.model_selection import train_test_split

def separa_treino_validacao_lfw(base_dir, proporcao_treino=0.8):
    caminhos_treino = []
    caminhos_validacao = []

    for nome_pessoa in os.listdir(base_dir):
        caminho_pessoa = os.path.join(base_dir, nome_pessoa)

        if not os.path.isdir(caminho_pessoa):
            continue

        imagens = [os.path.join(caminho_pessoa, img) for img in os.listdir(caminho_pessoa)
                   if img.lower().endswith(('.jpg', '.png', '.jpeg'))]

        total_imagens = len(imagens)

        if total_imagens not in [8, 10, 12]:
            continue

        imagens.sort()  # garantir consistência
        random.seed(42)  # garantir que o shuffle será o mesmo em todas as execuções do código
        random.shuffle(imagens)

        treino, validacao = train_test_split(imagens, train_size=proporcao_treino, shuffle=False)

        caminhos_treino.extend(treino)
        caminhos_validacao.extend(validacao)

    return caminhos_treino, caminhos_validacao


# Cria coleções

In [10]:
def popular_colecoes(client):
    imagens_treino, imagens_validacao = separa_treino_validacao_lfw("lfw_funneled")

    caracteristicas_lbp_treino = []
    caracteristicas_landmark_treino = []
    caracteristicas_cnn_treino = []

    for imagem in tqdm(imagens_treino, desc="Processando imagens de treinamento..."):
        caracteristicas_lbp_treino.append(extrai_caracteristicas_lbp(imagem))
        caracteristicas_landmark_treino.append(extrai_caracteristicas_landmarks(imagem))
        caracteristicas_cnn_treino.append(extrai_caracteristicas_cnn(imagem))

    caracteristicas_lbp_validacao = []
    caracteristicas_landmark_validacao = []
    caracteristicas_cnn_validacao = []

    for imagem in tqdm(imagens_validacao, desc="Processando imagens de validação..."):
        caracteristicas_lbp_validacao.append(extrai_caracteristicas_lbp(imagem))
        caracteristicas_landmark_validacao.append(extrai_caracteristicas_landmarks(imagem))
        caracteristicas_cnn_validacao.append(extrai_caracteristicas_cnn(imagem))
    
    client.upsert(collection_name="lbp_treinamento", points=caracteristicas_lbp_treino)
    client.upsert(collection_name="lbp_validacao", points=caracteristicas_lbp_validacao)
    client.upsert(collection_name="landmarks_treinamento", points=caracteristicas_landmark_treino)
    client.upsert(collection_name="landmarks_validacao", points=caracteristicas_landmark_validacao)
    client.upsert(collection_name="cnn_treinamento", points=caracteristicas_cnn_treino)
    client.upsert(collection_name="cnn_validacao", points=caracteristicas_cnn_validacao)

# Main

In [44]:
client = cria_cliente()

# Cria coleções para cada método de avaliação
cria_colecao(client, "lbp_treinamento", 54)
cria_colecao(client, "lbp_validacao", 54)
cria_colecao(client, "landmarks_treinamento", 136)
cria_colecao(client, "landmarks_validacao", 136)
cria_colecao(client, "cnn_treinamento", 128)
cria_colecao(client, "cnn_validacao", 128)

C:\Users\despinxz\AppData\Local\Temp\ipykernel_16228\814055332.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [47]:
imagens_treino, imagens_validacao = separa_treino_validacao_lfw("lfw_funneled")

caracteristicas_lbp_treino = []
caracteristicas_landmark_treino = []
caracteristicas_cnn_treino = []

for imagem in tqdm(imagens_treino, desc="Processando imagens de treinamento..."):
    caracteristicas_lbp_treino.append(extrai_caracteristicas_lbp(imagem))
    caracteristicas_landmark_treino.append(extrai_caracteristicas_landmarks(imagem))
    caracteristicas_cnn_treino.append(extrai_caracteristicas_cnn(imagem))

caracteristicas_lbp_validacao = []
caracteristicas_landmark_validacao = []
caracteristicas_cnn_validacao = []

for imagem in tqdm(imagens_validacao, desc="Processando imagens de validação..."):
    caracteristicas_lbp_validacao.append(extrai_caracteristicas_lbp(imagem))
    caracteristicas_landmark_validacao.append(extrai_caracteristicas_landmarks(imagem))
    caracteristicas_cnn_validacao.append(extrai_caracteristicas_cnn(imagem))

Processando imagens de treinamento...:  57%|█████▋    | 234/408 [02:11<01:37,  1.78it/s]


IndexError: list index out of range